In [1]:
import stravalib
import logging
from xml.dom import minidom
import dateutil
import numpy as np
import pandas as pd

import os

from datetime import datetime, timedelta, date


logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [2]:
#### Setting up strava API client

#Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client = stravalib.client.Client()
access_token = 'e3ccedc91fceef32077fbb31fc44676446d14bdd'

client.access_token = access_token
athlete = client.get_athlete()

## Enable accessing private activities
auth_url = client.authorization_url(client_id=19435, redirect_uri='http://localhost:8282/authorized', approval_prompt=u'auto', scope='view_private,write', state=None)
from IPython.core.display import display, HTML
display(HTML("""<a href="{}">{}</a>""".format(auth_url,auth_url)))

code = '6d057263b427852b0489af26e921f8fd25a78852'
access_token = client.exchange_code_for_token(client_id=19435, client_secret='45b776d5beceeb34c290b8a56bf9829d6d4ea5d7', code=code)

strava_client = stravalib.client.Client(access_token=access_token)
athlete = strava_client.get_athlete()
print('athlete name %s, athlete id %s.' %(athlete.firstname, athlete.id))

athlete name Brian, athlete id 3360678.


In [3]:
## Set up google sheets client, open worksheet
import pygsheets

gc = pygsheets.authorize(outh_file='client_secret.json', no_cache=True)

# Open spreadsheet and then workseet
sh = gc.open('Milburn Ultimate Scores')

In [4]:
## Get last entry from Raw Point Spreadsheet

wks = sh.worksheet_by_title('raw_points')

dates_recorded = [datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in wks.get_col(2) if d != '' and d != 'Start Time']
lap_start_date = max(dates_recorded) + timedelta(days=1)
# lap_start_date = dates_recorded[20]

from datetime import timedelta
lap_start_date = lap_start_date - timedelta(days=5)

lap_start_date

datetime.datetime(2018, 3, 15, 12, 58, 7)

In [5]:
runs = []
for activity in strava_client.get_activities(after=lap_start_date):
    if 'ltimate' in activity.name and activity.type == 'Run':
        a = strava_client.get_activity(activity.id)
        runs.append(a)

In [6]:
def get_strava_description(activity, p=False):
    new_activity = strava_client.get_activity(activity.id)
    try:
        scores, color = new_activity.description.split(' ')
        try:
            team_score, opponent_score = scores.split('-')
        except ValueError:
            team_score, opponent_score, color = None, None, None     
    except ValueError:
        scores = None
        team_score, opponent_score, color = None, None, None           

    try:
        color = color.lower()
    except AttributeError:
        color = None
        
    if p:
        print(new_activity.start_date)
        print(new_activity.description)
        print(team_score, opponent_score, color)
        
    return team_score, opponent_score, color

In [7]:
## Functions
def extract_events(run):
    lap_nums = []
    start_times = []
    elapsed_times = []
    for l in run.laps:
        try:
            lap_nums.append(int(l['name'].split(' ')[-1]))
            start_times.append(datetime.strptime(l['start_date_local'], '%Y-%m-%dT%H:%M:%SZ'))
            elapsed_times.append(timedelta(seconds=l['elapsed_time']))
        except TypeError:
            lap_nums.append(int(l.name.split(' ')[-1]))
            start_times.append(l.start_date_local)
            elapsed_times.append(l.elapsed_time)

    lap_nums = np.array(lap_nums)
    
    events = []
    for n, s, e in zip(np.diff(lap_nums), start_times, elapsed_times):
        events.append([n, s, e])
    
    return (events)
        
def process_events(events):
    
    event_lookup = [
        '',
        'team_point',
        'opponent_point',
        'my_point',
        'game',
        'game',
        'game'
    ]
    
    games = []
    game_num = 0
    added = False

    base_game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': 0, 'events':[], 'start_time':None, 'end_time':None}
    game = base_game

    for event in events:
        if game['start_time'] is None:
            game['start_time'] = event[1]

        game['end_time'] = event[1] + event[2]

        event_type = event_lookup[event[0]]
        game['events'].append((event_type, event[1], event[2]))


        if event_type == 'game':
            games.append(game)
            game_num += 1
            game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0, 'game_num': game_num, 'events':[], 'start_time':None, 'end_time':None}
            added = True

        elif event_type == 'my_point':
            game[event_type] += 1
            game['team_point'] += 1
            added = False

        else:
            game[event_type] += 1
            added = False


    if not added:
        games.append(game)
    
    
    ## Assign game winners
    for game in games:
        if game['team_point'] > game['opponent_point']:
            game['win'] = True
        else:
            game['win'] = False
            
    return games

In [8]:
## Compile raw points for export
games = []
all_data = []
for run in reversed(runs):
    team_wins, opponent_wins, color = get_strava_description(run, p=True)
    
    events = extract_events(run)

    point_df = pd.DataFrame(events, columns=['count', 'start_time', 'elapsed_time'])
    point_df['start_time'] = point_df['start_time'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    point_df['elapsed_time'] = point_df['elapsed_time'].apply(lambda x: x.seconds)
    
    data = point_df.sort_index(ascending=False).as_matrix().tolist()
    all_data = all_data + data
    
    
    

2018-03-19 17:15:05+00:00
0-2 white
0 2 white
2018-03-16 17:14:33+00:00
2-1 white
2 1 white


In [9]:
## Write raw points to gsheets
wks = sh.worksheet_by_title('raw_points')
wks.insert_rows(2, values=all_data, number=len(all_data))

In [37]:
## Get last entry from Data Spreadsheet

wks = sh.worksheet_by_title('game_summaries')

dates_recorded = [datetime.strptime(d, '%Y-%m-%d') for d in wks.get_col(1) if d != '' and d != 'Date']
lap_start_date = max(dates_recorded) + timedelta(days=1)
# lap_start_date = dates_recorded[20]
lap_start_date
# from datetime import timedelta
# lap_start_date = lap_start_date - timedelta(days=2)

datetime.datetime(2018, 3, 15, 12, 58, 7)

In [25]:
wks = sh.worksheet_by_title('raw_points')
col_names = wks.get_all_values()[1][0:3]

val_lists = wks.get_all_values()[2:]
val_lists = [v[0:3] for v in val_lists]
processed_raw_points = pd.DataFrame(val_lists, columns=col_names)

# Process
processed_raw_points['Start Time'] = processed_raw_points['Start Time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
processed_raw_points['Day'] = processed_raw_points['Start Time'].apply(lambda x: datetime(year=x.year, month=x.month, day=x.day))

processed_raw_points = processed_raw_points[processed_raw_points['Start Time'] > lap_start_date]
processed_raw_points

,Type,Start Time,Elapsed Time (Sec),Day
0,2,2018-03-19 12:54:17,242,2018-03-19
1,1,2018-03-19 12:51:32,150,2018-03-19
2,2,2018-03-19 12:49:27,123,2018-03-19
3,2,2018-03-19 12:48:05,81,2018-03-19
4,3,2018-03-19 12:46:47,77,2018-03-19
5,2,2018-03-19 12:45:02,102,2018-03-19
6,2,2018-03-19 12:43:31,90,2018-03-19
7,1,2018-03-19 12:41:53,96,2018-03-19
8,1,2018-03-19 12:39:26,148,2018-03-19
9,4,2018-03-19 12:38:56,25,2018-03-19


In [31]:
for day, pdf in processed_raw_points.groupby('Day', sort=False):
    print(day)
    events = pdf[['Type', 'Start Time', 'Elapsed Time (Sec)']].as_matrix().tolist()
    
    for e in events:
        print(e)
    current_games = process_events(events)
    
    for g in current_games:
        g['my_color'] = color
        g['team_wins'] = team_wins
        g['opponent_wins'] = opponent_wins
        

    games = games + current_games


2018-03-19 00:00:00
2018-03-16 00:00:00


[['4', Timestamp('2018-03-16 13:07:53'), '28'],
 ['1', Timestamp('2018-03-16 13:06:33'), '78'],
 ['2', Timestamp('2018-03-16 13:05:21'), '76'],
 ['2', Timestamp('2018-03-16 13:02:12'), '186'],
 ['1', Timestamp('2018-03-16 12:59:57'), '133'],
 ['3', Timestamp('2018-03-16 12:56:09'), '225'],
 ['2', Timestamp('2018-03-16 12:54:53'), '80'],
 ['2', Timestamp('2018-03-16 12:54:11'), '42'],
 ['1', Timestamp('2018-03-16 12:53:37'), '31'],
 ['1', Timestamp('2018-03-16 12:52:10'), '85'],
 ['4', Timestamp('2018-03-16 12:51:28'), '43'],
 ['2', Timestamp('2018-03-16 12:48:59'), '138'],
 ['1', Timestamp('2018-03-16 12:46:41'), '140'],
 ['1', Timestamp('2018-03-16 12:45:13'), '95'],
 ['2', Timestamp('2018-03-16 12:43:43'), '79'],
 ['3', Timestamp('2018-03-16 12:42:07'), '93'],
 ['2', Timestamp('2018-03-16 12:40:08'), '117'],
 ['2', Timestamp('2018-03-16 12:34:41'), '325'],
 ['2', Timestamp('2018-03-16 12:32:35'), '125'],
 ['4', Timestamp('2018-03-16 12:32:05'), '28'],
 ['3', Timestamp('2018-03-16 12:

2018-03-19 17:15:05+00:00
0-2 white
0 2 white
[1, datetime.datetime(2018, 3, 19, 12, 15, 5), datetime.timedelta(0, 119)]
[1, datetime.datetime(2018, 3, 19, 12, 17, 5), datetime.timedelta(0, 1)]
[1, datetime.datetime(2018, 3, 19, 12, 17, 9), datetime.timedelta(0, 102)]
[2, datetime.datetime(2018, 3, 19, 12, 18, 54), datetime.timedelta(0, 265)]
[2, datetime.datetime(2018, 3, 19, 12, 23, 16), datetime.timedelta(0, 298)]
[2, datetime.datetime(2018, 3, 19, 12, 28, 16), datetime.timedelta(0, 351)]
[1, datetime.datetime(2018, 3, 19, 12, 34, 8), datetime.timedelta(0, 176)]
[2, datetime.datetime(2018, 3, 19, 12, 37, 6), datetime.timedelta(0, 110)]
[4, datetime.datetime(2018, 3, 19, 12, 38, 56), datetime.timedelta(0, 25)]
[1, datetime.datetime(2018, 3, 19, 12, 39, 26), datetime.timedelta(0, 148)]
[1, datetime.datetime(2018, 3, 19, 12, 41, 53), datetime.timedelta(0, 96)]
[2, datetime.datetime(2018, 3, 19, 12, 43, 31), datetime.timedelta(0, 90)]
[2, datetime.datetime(2018, 3, 19, 12, 45, 2), datet

In [29]:
df = pd.DataFrame(games).dropna()

In [30]:
df['date'] = df.end_time.apply(lambda x: date(x.year, x.month, x.day))

df = df.set_index(['date', 'game_num'], drop=False)
df

end_time  \
date       game_num                       
2018-03-16 0        2018-03-16 12:32:33   
           1        2018-03-16 12:52:11   
           2        2018-03-16 13:08:21   
2018-03-19 0        2018-03-19 12:39:21   
           1        2018-03-19 12:58:27   

                                                                events  \
date       game_num                                                      
2018-03-16 0         [(opponent_point, 2018-03-16 12:14:33, 0:00:47...   
           1         [(opponent_point, 2018-03-16 12:32:35, 0:02:05...   
           2         [(team_point, 2018-03-16 12:52:10, 0:01:25), (...   
2018-03-19 0         [(team_point, 2018-03-19 12:15:05, 0:01:59), (...   
           1         [(team_point, 2018-03-19 12:39:26, 0:02:28), (...   

                     game_num my_color  my_point  opponent_point  \
date       game_num                                                
2018-03-16 0                0    white         1               4   
           1                1    white         1               5   
           2                2    white         1               3   
2018-03-19 0                0    white         0               4   
           1                1    white         1               5   

                    opponent_wins          start_time  team_point team_wins  \
date       game_num                                                           
2018-03-16 0                    1 2018-03-16 12:14:33           5         2   
           1                    1 2018-03-16 12:32:35           3         2   
           2                    1 2018-03-16 12:52:10           6         2   
2018-03-19 0                    2 2018-03-19 12:15:05           4         0   
           1                    2 2018-03-19 12:39:26           4         0   

                       win        date  
date       game_num                     
2018-03-16 0          True  2018-03-16  
           1         False  2018-03-16  
           2          True  2018-03-16  
2018-03-19 0         False  2018-03-19  
           1         False  2018-03-19

In [31]:
pdf = df

In [32]:
pdf['white_wins'] = None
pdf['color_wins'] = None
pdf['white_point'] = None
pdf['color_point'] = None
pdf['game_winner'] = None

In [33]:
for (date, game_num), row in pdf.iterrows():      
    
    if row.my_color == 'white':
        if row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'white_wins'] = pdf.loc[(date, game_num), 'team_wins']            
        pdf.loc[(date, game_num), 'color_wins'] = pdf.loc[(date, game_num), 'opponent_wins']            
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'opponent_point']

    else:
        if not row.win:
            pdf.loc[(date, game_num), 'game_winner'] = 'White'
        else:
            pdf.loc[(date, game_num), 'game_winner'] = 'Color'
        pdf.loc[(date, game_num), 'color_wins'] = pdf.loc[(date, game_num), 'team_wins']            
        pdf.loc[(date, game_num), 'white_wins'] = pdf.loc[(date, game_num), 'opponent_wins']    
        pdf.loc[(date, game_num), 'color_point'] = pdf.loc[(date, game_num), 'team_point']                
        pdf.loc[(date, game_num), 'white_point'] = pdf.loc[(date, game_num), 'opponent_point']


In [34]:
pdf

end_time  \
date       game_num                       
2018-03-16 0        2018-03-16 12:32:33   
           1        2018-03-16 12:52:11   
           2        2018-03-16 13:08:21   
2018-03-19 0        2018-03-19 12:39:21   
           1        2018-03-19 12:58:27   

                                                                events  \
date       game_num                                                      
2018-03-16 0         [(opponent_point, 2018-03-16 12:14:33, 0:00:47...   
           1         [(opponent_point, 2018-03-16 12:32:35, 0:02:05...   
           2         [(team_point, 2018-03-16 12:52:10, 0:01:25), (...   
2018-03-19 0         [(team_point, 2018-03-19 12:15:05, 0:01:59), (...   
           1         [(team_point, 2018-03-19 12:39:26, 0:02:28), (...   

                     game_num my_color  my_point  opponent_point  \
date       game_num                                                
2018-03-16 0                0    white         1               4   
           1                1    white         1               5   
           2                2    white         1               3   
2018-03-19 0                0    white         0               4   
           1                1    white         1               5   

                    opponent_wins          start_time  team_point team_wins  \
date       game_num                                                           
2018-03-16 0                    1 2018-03-16 12:14:33           5         2   
           1                    1 2018-03-16 12:32:35           3         2   
           2                    1 2018-03-16 12:52:10           6         2   
2018-03-19 0                    2 2018-03-19 12:15:05           4         0   
           1                    2 2018-03-19 12:39:26           4         0   

                       win        date white_wins color_wins white_point  \
date       game_num                                                        
2018-03-16 0          True  2018-03-16          2          1           5   
           1         False  2018-03-16          2          1           3   
           2          True  2018-03-16          2          1           6   
2018-03-19 0         False  2018-03-19          0          2           4   
           1         False  2018-03-19          0          2           4   

                    color_point game_winner  
date       game_num                          
2018-03-16 0                  4       White  
           1                  5       Color  
           2                  3       White  
2018-03-19 0                  4       Color  
           1                  5       Color

In [35]:
def merge_two_dicts(x, y):
    """Given two dicts, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

In [36]:
scores = []
for (date, game_num), game in pdf.iterrows():
    base_dict = dict(date=date, game_num=game_num, white_wins=game.white_wins, color_wins=game.color_wins, game_winner=game.game_winner)    
    scores.append(merge_two_dicts(base_dict, dict(team='white', team_score=game.white_point, my_score=game.my_point if game.my_color == 'white' else None)))
    scores.append(merge_two_dicts(base_dict, dict(team='color', team_score=game.color_point, my_score=game.my_point if game.my_color == 'colors' else None)))

score_df = pd.DataFrame(scores).set_index(['date', 'game_num', 'team'], drop=False)

In [37]:
out_df = score_df[['date', 'white_wins', 'color_wins', 'game_num', 'game_winner', 'team', 'team_score', 'my_score']].sort_index(ascending=False, level=0).fillna(value='')
out_df['date'] = out_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [38]:
out_df

date white_wins color_wins  game_num  \
date       game_num team                                                
2018-03-19 1        white  2018-03-19          0          2         1   
                    color  2018-03-19          0          2         1   
           0        white  2018-03-19          0          2         0   
                    color  2018-03-19          0          2         0   
2018-03-16 2        white  2018-03-16          2          1         2   
                    color  2018-03-16          2          1         2   
           1        white  2018-03-16          2          1         1   
                    color  2018-03-16          2          1         1   
           0        white  2018-03-16          2          1         0   
                    color  2018-03-16          2          1         0   

                          game_winner   team  team_score my_score  
date       game_num team                                           
2018-03-19 1        white       Color  white           4        1  
                    color       Color  color           5           
           0        white       Color  white           4        0  
                    color       Color  color           4           
2018-03-16 2        white       White  white           6        1  
                    color       White  color           3           
           1        white       Color  white           3        1  
                    color       Color  color           5           
           0        white       White  white           5        1  
                    color       White  color           4

In [39]:
data = out_df.as_matrix().tolist()
wks = sh.worksheet_by_title('game_summaries')
wks.insert_rows(2, values=data, number=len(data))